In [1]:
# Libraries
import pandas as pd

In [2]:
data = pd.read_excel("/Users/conny/Desktop/AlphaFold/WT untagged_vs_RNH1(collabo_Charles Bou Nader).xlsx")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1717 entries, 0 to 1716
Data columns (total 31 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Symbol color                                          1717 non-null   object 
 1   Symbol type                                           1717 non-null   int64  
 2   Symbol size                                           1717 non-null   int64  
 3   Show label                                            0 non-null      float64
 4   Protein.Ids                                           1717 non-null   object 
 5   Protein.Names                                         1716 non-null   object 
 6   Genes                                                 1717 non-null   object 
 7   First.Protein.Description                             1716 non-null   object 
 8   Student's T-test Significant RNH1-HTP_WT-Untagged     739 

In [3]:
# Extract the Protein ID from the column
protein_ids = data['Protein.Ids']
print(type(protein_ids))
print(protein_ids)



<class 'pandas.core.series.Series'>
0              Q04740
1              P32445
2       P05755;O13516
3       P0CX45;P0CX46
4       P0CX39;P0CX40
            ...      
1712           P45976
1713           O60200
1714           P38934
1715           P52891
1716           P48510
Name: Protein.Ids, Length: 1717, dtype: object


In [30]:
# Example code, trail for 1 protein
import requests, sys, json

params = {
  "fields": [
    "sequence"
  ]
}
headers = {
  "accept": "application/json"
}
base_url = "https://rest.uniprot.org/uniprotkb/Q04740"

response = requests.get(base_url, headers=headers, params=params)
if not response.ok:
  response.raise_for_status()
  sys.exit()

data = response.json()

# Full information
#print(json.dumps(data, indent=2))
# sequence = data.get("sequence", {}).get("value", None)
# print(sequence)

# If the sequence is not found, return an empty dictionary
# If the actual sequence string is not found, return None
sequence_info = data.get("sequence", {})
sequence_value = sequence_info.get("value", None)

# Print the sequence value if it exists
if sequence_value:
    print(sequence_value)
else:
    print("Sequence value not found.")

test_file = "single_sequence.fasta"
# Save results to file
with open(test_file, "w") as file:
      file.write(f">Q04740\n{sequence_value}\n")

MARQGNFYAVRKGRETGIYNTWNECKNQVDGYGGAIYKKFNSYEQAKSFLGQPNTTSNYGSSTHAGGQVSKPHTTQKRVHRRNRPLHYSSLTSSSACSSLSSANTNTFYSVKSNVPNIESKIFNNWKDCQAYVKHKRGITFKKFEDQLAAENFISGMSAHDYKLMNISKESFESKYKLSSNTMYNKSMNVYCDGSSFGNGTSSSRAGYGAYFEGAPEENISEPLLSGAQTNNRAEIEAVSEALKKIWEKLTNEKEKVNYQIKTDSEYVTKLLNDRYMTYDNKKLEGLPNSDLIVPLVQRFVKVKKYYELNKECFKNNGKFQIEWVKGHDGDPGNEMADFLAKKGASRR


In [5]:
# Extract the Protein ID from the column and put them in a txt file, one per line
protein_ids = data['Protein.Ids']
# Create output file
with open('protein_ids.txt', 'w') as f:
    for entry in protein_ids:
        # Split on semicolon in case of multiple IDs per row
        ids = entry.split(';')
        for protein_id in ids:
            # Write each ID on a new line after stripping whitespace
            f.write(f"{protein_id.strip()}\n")

print("Protein IDs saved to protein_ids.txt")

# There are repeated IDs, a total of 1993


Protein IDs saved to protein_ids.txt


In [35]:
# The  foolowing script already remove the repeated IDs, a total of 1862*2 lines.
import requests
import sys
import json
import pandas as pd

# Extract the Protein ID from the column
protein_ids = data['Protein.Ids']

# API Parameters
params = {"fields": ["sequence"]}
headers = {"accept": "application/json"}

# Output file
output_file = "unique_protein_sequences.fasta"

# Function to fetch amino acid sequence
def fetch_sequence(protein_id):
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}"
    response = requests.get(url, headers=headers, params=params)

    if response.ok:
        data = response.json()
        sequence_info = data.get("sequence", {})
        return sequence_info.get("value", None)  # Return sequence or None if not found
    else:
        print(f"Warning: Failed to fetch {protein_id}", file=sys.stderr)
        return None

# Process protein IDs and fetch sequences
results = {}
for entry in protein_ids:
    protein_ids = entry.split(";")  # Handle multiple protein IDs in a row
    for protein_id in protein_ids:
        protein_id = protein_id.strip()  # Remove whitespace
        if protein_id and protein_id not in results:  # Avoid redundant queries
            sequence = fetch_sequence(protein_id)
            if sequence:
                results[protein_id] = sequence
            else:
                results[protein_id] = "Sequence Not Found"

# Save results to file
with open(output_file, "w") as file:
    for protein_id, sequence in results.items():
        file.write(f">{protein_id}\n{sequence}\n")

print(f"Protein sequences saved to {output_file}")


Protein sequences saved to protein_sequences.fasta


In [ ]:
#